<img width="400" src="https://nomeroff.net.ua/images/logo/nomeroff_net.svg" alt="Nomeroff Net. Automatic numberplate recognition system" align="left"/>

## The example demonstrates license plate number detection.

Before runing this demo, please download models from [https://nomeroff.net.ua/models/](https://nomeroff.net.ua/models/) to **./models/** directory. 

In [1]:
import os
import pandas as pd

In [3]:
# Import all necessary libraries.
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt

In [12]:
from _paths import nomeroff_net_dir
from nomeroff_net.pipes.number_plate_localizators.yolo_v5_detector import Detector
from nomeroff_net.pipes.number_plate_keypoints_detectors.bbox_np_points import NpPointsCraft

from nomeroff_net.tools.image_processing import (
    get_cv_zone_rgb,
    convert_cv_zones_rgb_to_bgr,
    reshape_points)

In [5]:
detector = Detector()
detector.load()

np_points_craft = NpPointsCraft()
np_points_craft.load()

YOLOv5 🚀 v6.1-2-g4bab565 torch 1.9.0+cu102 CPU

Fusing layers... 
Model Summary: 213 layers, 7012822 parameters, 0 gradients


Loading weights from checkpoint (/mnt/data/var/www/nomeroff-net/tools/py/../../nomeroff_net/tools/../../data/./models/np_points_craft/craft_mlt/craft_mlt_25k_2020-02-16.pth)
Loading weights of refiner from checkpoint (/mnt/data/var/www/nomeroff-net/tools/py/../../nomeroff_net/tools/../../data/./models/np_points_craft/craft_refiner/craft_refiner_CTW1500_2020-02-16.pth)


In [6]:
%matplotlib inline
plt.rcParams["figure.figsize"] = (20,10)

In [7]:
root_dir = os.path.join(nomeroff_net_dir, './data/dataset/Detector/yolov5/autoria_numberplate_dataset_example/train/')

photos = pd.read_csv(os.path.join(nomeroff_net_dir,
                                  './data/dataset/Detector/yolov5/autoria_numberplate_dataset_example/train/photos_dataset.csv'))
photos = photos.set_index(['photoId'])

In [14]:
res_dir = os.path.join(nomeroff_net_dir, "./data/dataset/TextDetector/grab_res/")
os.makedirs(res_dir, exist_ok = False) 

for dir_name, subdir_list, file_list in os.walk(root_dir):
    for fname in file_list:
        #try:
            baseName = os.path.splitext(os.path.basename(fname))[0]
            img_path = os.path.join(dir_name, fname)
            try:
                img = mpimg.imread(img_path)   
            except IOError:
                print(f"[WARNING] {img_path} is not image")
                continue
            
            #print(img.reshape([1, *img.shape]))
            target_boxes = detector.detect_bbox(img)
            

            images_points, images_mline_boxes = np_points_craft.detect([[img, target_boxes]])
            all_points = images_points[0]
            all_points = [ps for ps in all_points if len(ps)]

            # cut zones
            to_show_zones = [get_cv_zone_rgb(img, reshape_points(rect, 1)) for rect in all_points]
            zones = convert_cv_zones_rgb_to_bgr(to_show_zones)

            if len(to_show_zones) > 1:
                areas = [zone.shape[0]*zone.shape[1] for zone in zones]
                zone = to_show_zones[np.argmax(areas)]
            else:
                zone = to_show_zones[0]

            text = photos.loc[baseName]['npText']
            
            imgpath = os.path.join(res_dir, "./{}_{}.png".format(baseName, text))
            mpimg.imsave(imgpath, zone)

        #except Exception as ex:
        #    print('[ERROR]', ex)

[WARNING] /mnt/data/var/www/nomeroff-net/tools/py/../.././data/dataset/Detector/yolov5/autoria_numberplate_dataset_example/train/photos_dataset.csv is not image
[WARNING] /mnt/data/var/www/nomeroff-net/tools/py/../.././data/dataset/Detector/yolov5/autoria_numberplate_dataset_example/train/via_region_data.json is not image
